In [1]:
import sys, time, random
import os
sys.path.append('../..')
import pybeamtools

[INFO ][MainThread][17:04:55.100 logging.py  23] Starting shared logging thread on PID 23644
[INFO ][log_listen][17:04:55.101 logging.py  32] Logging listener started on PID 23644
[INFO ][MainThread][17:04:55.103 logging.py  27] Logging setup finished


In [2]:
def time_fun():
    return time.time()*0.5

t = 0.0
def fixed_time():
    return t

In [3]:
from pybeamtools.sim.core import SimulationEngine, SignalEngineOptions
from pybeamtools.sim.pddevices2 import *

[INFO ][MainThread][17:04:55.690   utils.py 145] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO ][MainThread][17:04:55.690   utils.py 157] NumExpr defaulting to 8 threads.
[DEBUG][MainThread][17:04:56.073 _backend.py  35] Backend 'array' registered
[DEBUG][MainThread][17:04:56.075 _backend.py  35] Backend 'numpy' registered
[DEBUG][MainThread][17:04:56.075 _backend.py  47] Selecting backend: 'numpy'


In [4]:
sim = SimulationEngine(SignalEngineOptions(time_function=fixed_time))
sim.TRACE = True
sim.TIME_TRACE = True
ctx = SignalContext(se=sim)
echo1 = EchoDevice(EchoDeviceOptions(name='echo1', data={'TEST:ECHO:1':5}))
echo2 = EchoDevice(EchoDeviceOptions(name='echo2', data={'TEST:ECHO:2':15}))
echo3= EchoDevice(EchoDeviceOptions(name='echo3', data={'TEST:ECHO:3':25}))
echo4 = EchoDevice(EchoDeviceOptions(name='echo4', data={'TEST:ECHO:4':35}))

proxy1 = ProxyDevice(ProxyDeviceOptions(name='proxy1', channel_map={'TEST:PROXY:1':{'TEST:ECHO:1': TRIG.PROPAGATE}}))
proxy2 = ProxyDevice(ProxyDeviceOptions(name='proxy2', channel_map={'TEST:PROXY:2':{'TEST:ECHO:2': TRIG.IGNORE}}))
proxy3 = ProxyDevice(ProxyDeviceOptions(name='proxy3', channel_map={'TEST:PROXY:3':{'TEST:PROXY:1': TRIG.PROPAGATE}}))
proxy4 = ProxyDevice(ProxyDeviceOptions(name='proxy4', channel_map={'TEST:PROXY:4':{'TEST:PROXY:1': TRIG.PROPAGATE}}))

[DEBUG][MainThread][17:04:56.143    core.py 697] Starting update thread
[DEBUG][update_thr][17:04:56.144    core.py 546] Update thread (id 16916) on


In [5]:
for dev in [echo1, echo2, echo3, echo4]:
    sim.add_device(dev)

[DEBUG][MainThread][17:04:56.213    core.py 201] Added device (echo1) (device.channel_map={'TEST:ECHO:1': {}})
[DEBUG][MainThread][17:04:56.214    core.py 201] Added device (echo2) (device.channel_map={'TEST:ECHO:2': {}})
[DEBUG][MainThread][17:04:56.214    core.py 201] Added device (echo3) (device.channel_map={'TEST:ECHO:3': {}})
[DEBUG][MainThread][17:04:56.215    core.py 201] Added device (echo4) (device.channel_map={'TEST:ECHO:4': {}})


In [6]:
sim.enable_device(echo1)

[DEBUG][MainThread][17:04:56.304    core.py 506] Future event added
[DEBUG][MainThread][17:04:56.304    core.py 215] E0| waiting for futures
[DEBUG][update_thr][17:04:56.304    core.py 554] Processing E0:0 ev.op=<OP.ENABLE: 3> ev.data={}
[DEBUG][update_thr][17:04:56.305    core.py 565] E0:0 | Enable (dn='echo1') at (0.0)
[DEBUG][update_thr][17:04:56.306    core.py 499] Future callback for ev.txid=0
[DEBUG][MainThread][17:04:56.306    core.py 217] done={<Future at 0x1aae5389f70 state=finished returned bool>} not_done=set()
[DEBUG][MainThread][17:04:56.307    core.py 220] Device (echo1) enabled


In [7]:
sim.update_thread_running

True

In [8]:
sim.read_channel('TEST:ECHO:1')

[DEBUG][MainThread][17:04:56.437    core.py 407] Read event (TEST:ECHO:1) added
[DEBUG][update_thr][17:04:56.437    core.py 554] Processing E1000:1000 ev.op=<OP.READ: 1> ev.data={'TEST:ECHO:1': None}
[DEBUG][update_thr][17:04:56.438    core.py 605] E1000:1000 | Read (TEST:ECHO:1=5) at (0.0)
[DEBUG][update_thr][17:04:56.438    core.py 400] Read callback (TEST:ECHO:1)=({'TEST:ECHO:1': 5})
[DEBUG][MainThread][17:04:56.439    core.py 410] Read event (results={'TEST:ECHO:1': 5} (success=True)


5

In [9]:
var = 'TEST:GEN1'
data = {var:1.5}
def update_fun(t_sched, t_run, dep_data, self):
    return data

def read_fun(t_sched, t_run, channel_name, self):
    return data[var]

def write_fun(t_sched, t_run, value_dict, self):
    data[var] = value_dict[var]

def startup_fun(dev):
    import threading
    def updater():
        while True:
            data[var] = random.random()*10
            #print(f'Issuing update {data=}')
            dev.ctx.issue_update(dev, data)
            time.sleep(20)
    th = threading.Thread(target=updater, name='updater')
    th.daemon = True
    th.start()
gopt = GenericDeviceOptions(name='gen', update_fun=update_fun, read_fun=read_fun, write_fun=write_fun, startup_fun=startup_fun, channel_map={var:{}})
generic1 = GenericDevice(ctx, options=gopt)

TypeError: __init__() got multiple values for argument 'options'

In [ ]:
sim.dev_channels_map

In [ ]:
proxy1 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy1', channel_map={'TEST:PROXY:1':{var: TRIGSPEC.PROPAGATE}}))
proxy2 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy2', channel_map={'TEST:PROXY:2':{'TEST:ECHO:2': TRIGSPEC.IGNORE}}))
proxy3 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy3', channel_map={'TEST:PROXY:3':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}}))
proxy4 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy4', channel_map={'TEST:PROXY:4':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}}))

In [ ]:
proxy1.state

In [ ]:
generic1.state

In [ ]:
sim.latest_data

In [ ]:
sim.enable_device(echo)
sim.enable_device(echo2)
sim.enable_device(proxy1)

In [ ]:
sim.latest_data

In [ ]:
sim.write_channel('TEST:ECHO:1', 3.0)

In [ ]:
sim.latest_data

In [ ]:
print(sim.channels_map_chain)
print(sim.channels_dep_chain)
print(sim.next_periodic_read_time)

In [ ]:
import typing
field_to_type = typing.get_type_hints(echo.options)

In [ ]:
field_to_type

In [ ]:
sim.latest_data